In [1]:
import os
import requests
from selenium import webdriver
import json

офигенный автоматический способ получить токен

In [ ]:
# Создаем объект драйвера

driver = webdriver.Firefox()

# Переходим по ссылке.
# client_id - идентификатор созданного нами приложения
# scope - права доступа
driver.get("http://api.vkontakte.ru/oauth/authorize?"
           "client_id=4591034&scope=audio"
           "&redirect_uri=http://api.vk.com/blank.html"
           "&display=page&response_type=token")

user = "email/phone"
password = "password"

# Находим элементы формы и вводим данные для авторизации
user_input = driver.find_element_by_name("email")
user_input.send_keys(user)
password_input = driver.find_element_by_name("pass")
password_input.send_keys(password)

# Нажимаем на кнопку
submit = driver.find_element_by_id("install_allow")
submit.click()

# Получаем необходимые данные для выполнения запросов к api
current = driver.current_url
access_list = (current.split("#"))[1].split("&")
access_token = (access_list[0].split("="))[1] # acces_token
expires_in = (access_list[1].split("="))[1] # срок времени действия токена
user_id = (access_list[2].split("="))[1] # id нашей учетной записи в ВК
# Закрываем окно браузера

driver.close()

In [ ]:
# Принт для дебага
print "Connecting"
# Адрес запроса
url = "https://api.vkontakte.ru/method/" \
      "audio.get?uid=" + user_id +\
      "&access_token=" + access_token
# Создаем листы для хранения данных
artists_list = []
titles_list = []
links_list = []
# счетчик для дебага и перехода по элементам листов
number = 0
# Читаем ответ сервера и сохраняем в переменную
page = requests.get(url)
html = page.text

In [ ]:
my_dict = json.loads(html) # используем loads()

In [ ]:
for i in my_dict['response']:
    artists_list.append(i['artist'])
    titles_list.append(i['title'])
    links_list.append(i['url'])
    number += 1

In [ ]:
# Создаем директорию, если она не была создана ранее

path = "downloads"

if not os.path.exists(path):
    os.makedirs(path)
# Принт для дебага
print "Need to download: ", number

# Процесс скачивания файлов

for i in range(0, number):

# Путь по которому будет храниться/скачиваться конкретная аудиозапись
    new_filename = path+"/"+artists_list[i] + " - " + titles_list[i] + ".mp3"
    print "Downloading: ", new_filename, i
# Проверка на существующий файл
    if not os.path.exists(new_filename):
# Сама загрузка файла, отсекаем из ссылки все аргументы и указываем путь куда скачивать
        with open(new_filename, "wb") as out:
            response = requests.get(links_list[i].split("?")[0])
            out.write(response.content)

print "Download complete."